# Data Issue Analysis

Data had loads of redundant rows and other minor easy to fix data issues, which are already taken care of while doing ETL on the data.<br>
This doc covers some major issues that still remain.<br>
Most of the Analysis is done in SQL by connecting to Postgres

## Brands
1. Brands missing brand category in many places.
2. Out of 1167 brands row, 234 do not contain any brandcode
3. From 568 distinct barcodes in reciepts 552 are not present in brands i.e. 97% of expected barcodes are missing in brands data.
4. The above issue makes recieptsItems to brand mapping nearly impossible. We can use brandcode, but brandcode is not unique since one brandcode can have multiple barcodes also since many rows dont have brandcodes either.
5. Its also difficult to seperate out brand and barcode in one to many relation due to above inconsistencies.
4. Brands has 7 barcodes which coresponds to two different brands at the same time. (This is a critical flaw)

## Users
1. Almost half the data in Users was redundant, with only 212 records after removing duplicates
2. Multiple users are missing signUpSource, lastlogin and state
3. Users data is missing 117 unique users which are referenced in Reciepts. Which is 45% of the total(258) unique userids in Receipts

## Reciepts
1. Reciepts have 6941 unique receiptItems out of which only 3090 have barcode and only 2600 have brandcode. Due to this its really difficult to map recieptItems to brand. 
2. Receipt and RecieptItems are missing multiple crucial values like prices, dates and bonus in many of the rows. and missing no crucial values in most of the rows.
3. Receipts references multiple brandCodes and barcodes not present in Brands

# Analysis Code

In [101]:
#Helper code to connect to filtered data on postgres
import psycopg2
import json
import pandas as pd
from sqlalchemy import create_engine

with open("config/config.json", 'r') as file:
    # Parse the JSON data into a dictionary
    config = json.load(file)
    connection = psycopg2.connect(user=config["user"],
                                  password=config["password"],
                                  host=config["host"],
                                  port=config["port"],
                                  database=config["database"])
def queryResults(query,connection=connection):
    with open("config/config.json", 'r') as file:
        # Parse the JSON data into a dictionary
        config = json.load(file)
        connection = psycopg2.connect(user=config["user"],
                                      password=config["password"],
                                      host=config["host"],
                                      port=config["port"],
                                      database=config["database"])
        cursor = connection.cursor()

        cursor.execute(query, ('your_value',))

        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        cursor.close()
        connection.close()
        df = pd.DataFrame(rows,columns=column_names)
        return df

In [102]:
# Find missing values in Brands
query="""
SELECT * FROM brands
"""
queryResults(query).info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   _id        1167 non-null   object 
 1   barcode    1167 non-null   object 
 2   brandcode  933 non-null    object 
 3   catid      1012 non-null   float64
 4   cpgid      0 non-null      object 
 5   cpgref     0 non-null      object 
 6   topbrand   555 non-null    object 
 7   name       1167 non-null   object 
dtypes: float64(1), object(7)
memory usage: 73.1+ KB


In [103]:
# Find missing values in BrandsCategories
query="""
SELECT * FROM brandCategories
"""
queryResults(query).info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           23 non-null     int64 
 1   categorycode  23 non-null     object
 2   category      23 non-null     object
dtypes: int64(1), object(2)
memory usage: 680.0+ bytes


In [104]:
# Find missing values in Brands
query="""
SELECT 
    COUNT(_id) AS id_count,
    COUNT(_id)- COUNT(barcode) AS null_barcode_count,
    COUNT(_id) - COUNT(brandcode) AS null_brandcode_count
FROM brands;
"""
queryResults(query)

,id_count,null_barcode_count,null_brandcode_count
0,1167,0,234


In [105]:
# Find brandcodes which are present in receiptItems but not in brands
q1='''
SELECT count(distinct barcode) as distinct_barcode_not_in_brands
FROM receiptItems
WHERE barcode NOT IN (SELECT barcode FROM brands);
'''
queryResults(q1)

,distinct_barcode_not_in_brands
0,552


In [106]:
# Find brandcodes,barcodes which are present in receiptItems but not in brands
q1='''
WITH query1 AS (
    SELECT count(distinct barcode) as distinct_barcode_not_in_brands, ROW_NUMBER() OVER () AS row_num
    FROM receiptItems
    WHERE barcode NOT IN (SELECT barcode FROM brands)
),
query2 AS (
    SELECT count(distinct brandcode) as distinct_brandcode_not_in_brands, ROW_NUMBER() OVER () AS row_num
    FROM receiptItems
    WHERE brandcode NOT IN (SELECT brandcode FROM brands)
),
query3 AS (
    SELECT count(distinct barcode) as distinct_barcode_in_reciepts,count(distinct brandcode) as distinct_brandcode_in_reciepts, ROW_NUMBER() OVER () AS row_num
    FROM receiptItems
)
SELECT q3.distinct_barcode_in_reciepts,q1.distinct_barcode_not_in_brands,q3.distinct_brandcode_in_reciepts, q2.distinct_brandcode_not_in_brands
FROM query1 q1
JOIN query2 q2 ON q1.row_num = q2.row_num
JOIN query3 q3 ON q1.row_num =q3.row_num
;'''
queryResults(q1)

,distinct_barcode_in_reciepts,distinct_barcode_not_in_brands,distinct_brandcode_in_reciepts,distinct_brandcode_not_in_brands
0,568,552,227,0


In [107]:
# Find barcodes which point to different brands
q1="""
SELECT barcode, COUNT(*) AS count
FROM brands
GROUP BY barcode
HAVING COUNT(*) > 1;
"""
queryResults(q1)

,barcode,count
0,511111605058,2
1,511111204923,2
2,511111704140,2
3,511111504788,2
4,511111504139,2
5,511111305125,2
6,511111004790,2


In [118]:
q1="""
SELECT b.*
FROM brands b
JOIN (
    SELECT barcode
    FROM brands
    GROUP BY barcode
    HAVING COUNT(*) > 1
) dup ON b.barcode = dup.barcode
order by barcode;
"""
queryResults(q1)

,_id,barcode,brandcode,catid,cpgid,cpgref,topbrand,name
0,5cdacd63166eb33eb7ce0fa8,511111004790,BITTEN,4.0,None,None,None,Bitten Dressing
1,5c409ab4cd244a3539b84162,511111004790,ALEXA,1.0,None,None,True,alexa
2,5d6027f46d5f3b23d1bc7906,511111204923,CHESTERS,14.0,None,None,None,CHESTER'S
3,5c45f91b87ff3552f950f027,511111204923,0987654321,13.0,None,None,True,Brand1
4,5c4699f387ff3577e203ea29,511111305125,CHRISIMAGE,11.0,None,None,None,Chris Image Test
5,5d642d65a3a018514994f42d,511111305125,511111305125,6.0,None,None,None,Rachael Ray Everyday
6,5a8c33f3e4b07f0a2dac8943,511111504139,PACE,13.0,None,None,False,Pace
7,5a7e0604e4b0aedb3b84afd3,511111504139,CHRISXYZ,2.0,None,None,None,Chris Brand XYZ
8,5ccb2ece166eb31bbbadccbe,511111504788,PIONEER WOMAN,4.0,None,None,None,The Pioneer Woman
9,5c408e8bcd244a1fdb47aee7,511111504788,TEST,1.0,None,None,None,test


In [108]:
query="""
SELECT * FROM users
"""
queryResults(query).info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   _id           212 non-null    object        
 1   state         206 non-null    object        
 2   createddate   212 non-null    datetime64[ns]
 3   lastlogin     172 non-null    datetime64[ns]
 4   roleid        212 non-null    int64         
 5   active        212 non-null    bool          
 6   signupsource  207 non-null    object        
dtypes: bool(1), datetime64[ns](2), int64(1), object(3)
memory usage: 10.3+ KB


In [109]:
query="""
SELECT * FROM userroles
"""
queryResults(query)

,_id,role
0,1,consumer
1,2,fetch-staff


In [110]:
query="""
SELECT DISTINCT r.userid
FROM Receipts r
LEFT JOIN users u ON r.userid = u._id
WHERE u._id IS NULL;
"""
queryResults(query)

,userid
0,601890b2c8b50e11d8454e82
1,5ff73be9eb7c7d31ca8a45bc
2,600ba68e7d983a124e9ae1d3
3,5ff4ce68c1e2d0121a9b3022
4,600f489d6fd0dc1768a35a88
...,...
112,5fa5ad376a26f611e71ab5ef
113,5f9c74f7c88c1415cbddb839
114,6008412f6e64697abedcd5d5
115,5ffc8cb304929111f6e9229d


In [111]:
query="""
SELECT count(DISTINCT userid) as unique_userids_in_reciepts
FROM Receipts
"""
queryResults(query)

,unique_userids_in_reciepts
0,258


In [112]:
query="""
SELECT * FROM Receipts
"""
queryResults(query).info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   _id                      1119 non-null   object        
 1   bonuspointsearned        544 non-null    float64       
 2   bonuspointsearnedreason  544 non-null    object        
 3   createdate               1119 non-null   datetime64[ns]
 4   datescanned              1119 non-null   datetime64[ns]
 5   finisheddate             568 non-null    datetime64[ns]
 6   modifydate               1119 non-null   datetime64[ns]
 7   pointsawardeddate        537 non-null    datetime64[ns]
 8   pointsearned             609 non-null    float64       
 9   purchasedate             671 non-null    datetime64[ns]
 10  purchaseditemcount       635 non-null    float64       
 11  rewardsreceiptstatus     1119 non-null   object        
 12  totalspent               684 non-n

In [113]:
query="""
SELECT * FROM ReceiptItems
"""
queryResults(query).info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 37 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   _id                                 6941 non-null   object 
 1   recieptid                           6941 non-null   object 
 2   brandid                             82 non-null     object 
 3   barcode                             3090 non-null   object 
 4   description                         6560 non-null   object 
 5   finalprice                          6767 non-null   float64
 6   itemprice                           6767 non-null   float64
 7   needsfetchreview                    813 non-null    object 
 8   partneritemid                       6941 non-null   object 
 9   preventtargetgappoints              358 non-null    object 
 10  quantitypurchased                   6767 non-null   float64
 11  userflaggedbarcode                  337 non

In [114]:
query="""
SELECT barcode,brandcode FROM ReceiptItems
"""
queryResults(query)

,barcode,brandcode
0,4011,None
1,4011,None
2,028400642255,None
3,None,None
4,4011,None
...,...,...
6936,B07BRRLSVC,None
6937,B076FJ92M4,None
6938,B07BRRLSVC,None
6939,B076FJ92M4,None
